In [1]:
%pip install --quiet pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


Now we can read in data from an Excel file, using pandas.

In [2]:
import pandas
data = pandas.read_excel('Students.xlsx', sheet_name=None)

The file has two worksheets in it, one with student information.

In [3]:
students = data['Students']
students

,Student ID,First Name,Last Name,GPA,Major
0,1,Emma,Johnson,3.8,CS
1,2,Liam,Smith,3.5,CS
2,3,Olivia,Williams,3.9,CS
3,4,Noah,Jones,3.7,Maths
4,5,Ava,Brown,3.2,CS
5,6,Isabella,Davis,3.6,CS
6,7,Sophia,Miller,2.9,CS
7,8,Mia,Wilson,3.3,Maths
8,9,Harper,Moore,3.1,CS
9,10,Benjamin,Taylor,3.8,Maths


And another one that has the "forbidden pairs" in it.

In [4]:
forbiddens = data['Forbidden pairs']
forbiddens

,Student 1,Student 2
0,1,3
1,1,4
2,7,13
3,8,17
4,3,21


Let's preprocess the data and prepare it for the model.

In [5]:
import math

names = {}
gpas = {}
majors = {}
pots = {}

# for line in students.values.tolist():
#     [id, name, surname, gpa, major] = line
#     gpas[id] = int(gpa * 10)
#     majors[id] = 0 if major == "CS" else 1

studentsSortedByGPA = sorted(students.values.tolist(),
                                key = lambda line: line[3])

for index, line in enumerate(studentsSortedByGPA):
    [id, name, surname, gpa, major] = line
    gpas[id] = int(gpa * 10)
    majors[id] = 0 if major == "CS" else 1
    pots[id] = 1 + math.floor(index / 4)
    names[id] = f'[{id}] {name} {surname} ({major} - {gpa} - pot {pots[id]})'

nbStudents = len(students)


Installing and loading the Conjure notebook extension

In [6]:
%pip install --quiet git+https://github.com/conjure-cp/conjure-notebook.git@v0.0.8
%load_ext conjure

Note: you may need to restart the kernel to use updated packages.


<IPython.core.display.Javascript object>

Conjure extension is loaded.
For usage help run: %conjure_help


This extension contains a few _magic_ commands, for example `%%conjure`. Also see `%conjure_help`.


These are the parameters of our model.

In [7]:
%%conjure

given nbStudents : int
given gpas : function (total) int(1..nbStudents) --> int
given majors : function (total) int(1..nbStudents) --> int(0,1)
given pots : function (total) int(1..nbStudents) --> int(1..6)


```json
{}
```

And we are trying to find a partitioning of the students.

A partition is a _mutually exclusive_ and _collectively exhaustive_ collection of parts.

Here, we say each part will contain 6 items it it. We know there are 6 pots in this problem. We could have easily parameterised the model over the number of pots, too.

This implies that there will be 4 parts. We can provide this information as well, using `numParts 4` as a domain attribute. We will leave this out for now.

In [8]:
%%conjure+

find groups : partition (partSize 6) from int(1..nbStudents)


```json
{"groups": [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24]]}
```

We have a solution now!

It is a solution to an incomplete model however. It doesn't enfore any of our constraints yet. Let's format it sligtly better using Python.

In [9]:
for n, group in enumerate(groups):
    print(f'# Group {n+1}')
    for s in group:
        print(names[s])
    print()

# Group 1
[1] Emma Johnson (CS - 3.8 - pot 5)
[2] Liam Smith (CS - 3.5 - pot 3)
[3] Olivia Williams (CS - 3.9 - pot 6)
[4] Noah Jones (Maths - 3.7 - pot 4)
[5] Ava Brown (CS - 3.2 - pot 2)
[6] Isabella Davis (CS - 3.6 - pot 4)

# Group 2
[7] Sophia Miller (CS - 2.9 - pot 1)
[8] Mia Wilson (Maths - 3.3 - pot 3)
[9] Harper Moore (CS - 3.1 - pot 2)
[10] Benjamin Taylor (Maths - 3.8 - pot 5)
[11] Elijah Anderson (CS - 3.9 - pot 6)
[12] Oliver Thomas (Maths - 2.7 - pot 1)

# Group 3
[13] Mei Ling Chen (CS - 3.7 - pot 5)
[14] Aarav Patel (CS - 3.6 - pot 4)
[15] Layla Rahman (CS - 3.2 - pot 2)
[16] Michael Martin (Maths - 2.9 - pot 2)
[17] Emily Thompson (CS - 3.3 - pot 3)
[18] Zara Abbas (CS - 2.6 - pot 1)

# Group 4
[19] Alexander Martinez (CS - 3.8 - pot 6)
[20] Aarav Sharma (Maths - 3.9 - pot 6)
[21] Daniel Clark (CS - 2.8 - pot 1)
[22] Jing Wang (CS - 3.7 - pot 5)
[23] Fatima Ali (Maths - 3.6 - pot 4)
[24] Ahmed Khalid (CS - 3.2 - pot 3)



This is a useful bit of functionality, let's define a Python function that can print the solution this way.

In [10]:
def printSolution():
    # this is not best practice in general Python programming
    # groups is a global variable
    # however, it's probably fine in a notebook environment
    for n, group in enumerate(groups):
        print(f'# Group {n+1}')
        for s in group:
            print(names[s])
        print()

OK, let's focus on the intermediate solution again. In group 1, we have two people from pot 4. This is not good.

Let's enforce the condition that there has to be a member from each pot in every part of the partition.

In [11]:
%%conjure+

$ one member from each pot
such that
    forAll group in groups .
        forAll pot : int(1..6) .
            exists s in group .
                pots(s) = pot


```json
{"groups": [[1, 2, 3, 4, 5, 7], [6, 8, 9, 10, 11, 12], [13, 14, 15, 17, 18, 19], [16, 20, 21, 22, 23, 24]]}
```

In [12]:
printSolution()

# Group 1
[1] Emma Johnson (CS - 3.8 - pot 5)
[2] Liam Smith (CS - 3.5 - pot 3)
[3] Olivia Williams (CS - 3.9 - pot 6)
[4] Noah Jones (Maths - 3.7 - pot 4)
[5] Ava Brown (CS - 3.2 - pot 2)
[7] Sophia Miller (CS - 2.9 - pot 1)

# Group 2
[6] Isabella Davis (CS - 3.6 - pot 4)
[8] Mia Wilson (Maths - 3.3 - pot 3)
[9] Harper Moore (CS - 3.1 - pot 2)
[10] Benjamin Taylor (Maths - 3.8 - pot 5)
[11] Elijah Anderson (CS - 3.9 - pot 6)
[12] Oliver Thomas (Maths - 2.7 - pot 1)

# Group 3
[13] Mei Ling Chen (CS - 3.7 - pot 5)
[14] Aarav Patel (CS - 3.6 - pot 4)
[15] Layla Rahman (CS - 3.2 - pot 2)
[17] Emily Thompson (CS - 3.3 - pot 3)
[18] Zara Abbas (CS - 2.6 - pot 1)
[19] Alexander Martinez (CS - 3.8 - pot 6)

# Group 4
[16] Michael Martin (Maths - 2.9 - pot 2)
[20] Aarav Sharma (Maths - 3.9 - pot 6)
[21] Daniel Clark (CS - 2.8 - pot 1)
[22] Jing Wang (CS - 3.7 - pot 5)
[23] Fatima Ali (Maths - 3.6 - pot 4)
[24] Ahmed Khalid (CS - 3.2 - pot 3)



Better.

Now each group has exactly one member from each pot.

But group 1 has a single Maths major in it. Noah might feel lonely, let's enforce the condition that there cannot be a single person from any particular major.

This means a group can contain all CS or all Maths students, but if there is a mixture there will be at least 2 students from each major.

In [13]:
%%conjure+

$ no single major
such that
    forAll group in groups .
        forAll major : int(0,1) .
            sum([ toInt(majors(s) = major) | s <- group ]) != 1

```json
{"groups": [[1, 2, 3, 4, 5, 12], [6, 7, 8, 9, 10, 11], [13, 14, 15, 17, 18, 19], [16, 20, 21, 22, 23, 24]]}
```

In [14]:
printSolution()

# Group 1
[1] Emma Johnson (CS - 3.8 - pot 5)
[2] Liam Smith (CS - 3.5 - pot 3)
[3] Olivia Williams (CS - 3.9 - pot 6)
[4] Noah Jones (Maths - 3.7 - pot 4)
[5] Ava Brown (CS - 3.2 - pot 2)
[12] Oliver Thomas (Maths - 2.7 - pot 1)

# Group 2
[6] Isabella Davis (CS - 3.6 - pot 4)
[7] Sophia Miller (CS - 2.9 - pot 1)
[8] Mia Wilson (Maths - 3.3 - pot 3)
[9] Harper Moore (CS - 3.1 - pot 2)
[10] Benjamin Taylor (Maths - 3.8 - pot 5)
[11] Elijah Anderson (CS - 3.9 - pot 6)

# Group 3
[13] Mei Ling Chen (CS - 3.7 - pot 5)
[14] Aarav Patel (CS - 3.6 - pot 4)
[15] Layla Rahman (CS - 3.2 - pot 2)
[17] Emily Thompson (CS - 3.3 - pot 3)
[18] Zara Abbas (CS - 2.6 - pot 1)
[19] Alexander Martinez (CS - 3.8 - pot 6)

# Group 4
[16] Michael Martin (Maths - 2.9 - pot 2)
[20] Aarav Sharma (Maths - 3.9 - pot 6)
[21] Daniel Clark (CS - 2.8 - pot 1)
[22] Jing Wang (CS - 3.7 - pot 5)
[23] Fatima Ali (Maths - 3.6 - pot 4)
[24] Ahmed Khalid (CS - 3.2 - pot 3)



We have a list of people who cannot be in the same group for some reason. Let's remember who these people were.

In [15]:
forbiddens = forbiddens.values.tolist()
forbiddens

[[1, 3], [1, 4], [7, 13], [8, 17], [3, 21]]

Not good. 1 and 3 (Emma and Olivia) are not supposed to be in the same group, but they are.

Let's add another constraint. Pairs of people listed together in the forbidden list must be in different parts of the partition.

You can learn more about the apart constraint expression later. For now, the short version is that it takes two arguments, a set of values and a partition. Items in the set of values must not be together in the same part. Exactly what we need.

In [16]:
%%conjure+

given forbiddens : set of (int, int)
such that
    forAll (a,b) in forbiddens .
        apart({a,b}, groups)

```json
{"groups": [[1, 2, 5, 6, 7, 11], [3, 4, 8, 9, 10, 12], [13, 14, 15, 17, 18, 19], [16, 20, 21, 22, 23, 24]]}
```

In [17]:
printSolution()

# Group 1
[1] Emma Johnson (CS - 3.8 - pot 5)
[2] Liam Smith (CS - 3.5 - pot 3)
[5] Ava Brown (CS - 3.2 - pot 2)
[6] Isabella Davis (CS - 3.6 - pot 4)
[7] Sophia Miller (CS - 2.9 - pot 1)
[11] Elijah Anderson (CS - 3.9 - pot 6)

# Group 2
[3] Olivia Williams (CS - 3.9 - pot 6)
[4] Noah Jones (Maths - 3.7 - pot 4)
[8] Mia Wilson (Maths - 3.3 - pot 3)
[9] Harper Moore (CS - 3.1 - pot 2)
[10] Benjamin Taylor (Maths - 3.8 - pot 5)
[12] Oliver Thomas (Maths - 2.7 - pot 1)

# Group 3
[13] Mei Ling Chen (CS - 3.7 - pot 5)
[14] Aarav Patel (CS - 3.6 - pot 4)
[15] Layla Rahman (CS - 3.2 - pot 2)
[17] Emily Thompson (CS - 3.3 - pot 3)
[18] Zara Abbas (CS - 2.6 - pot 1)
[19] Alexander Martinez (CS - 3.8 - pot 6)

# Group 4
[16] Michael Martin (Maths - 2.9 - pot 2)
[20] Aarav Sharma (Maths - 3.9 - pot 6)
[21] Daniel Clark (CS - 2.8 - pot 1)
[22] Jing Wang (CS - 3.7 - pot 5)
[23] Fatima Ali (Maths - 3.6 - pot 4)
[24] Ahmed Khalid (CS - 3.2 - pot 3)



Looks good!

This is not a unique solution.

More constraints can be added.

An objective can be added (for example one to do with average gpa of a group?).

Or more solutions can be enumerated.

Stay tuned...